# Problem distorzije kod fotografija

Do distorzije dolazi kada pokušavamo da kopiramo 3D svet u 2D sliku kamerom. Neki od primera distorzije možemo videti na slikama ispod.
<br>
<br>
<img src="assets/example0_und.png" width="400px">
<br>
<img src="assets/example_und.png" width="2500px">

Na ovim iskrivljenim slikama možete videti da su ivice traka savijene i nekako zaobljene ili ispružene prema spolja. Naš prvi korak u analizi kamere je da poništimo ovu distorziju kako bismo iz njih mogli da dobijemo tačne i korisne informacije.

# Zašto se javlja distorzija?

Evo jednostavnog modela fotoaparata koji se zove **pinhole model**.
<br>
<img src="assets/why_dist.png" width="700px">
<br>
Kada kamera gleda predmet, ona gleda svet sličan onome kao naše oči. Usredsređivanjem svetlosti koja se odbija od objekata u svetu. U ovom slučaju, iako je mali otvor, fotoaparat fokusira svetlo koje se odbija na 3D saobraćajni znak i formira 2D sliku sa zadnje strane kamere.
<br>
<img src="assets/why_dist2.png" width="700px">
<br>
U matematici, transformacija iz 3D tačaka, $P(X,Y,Z) \rightarrow p(x,y)$ vrši se matricom transformacije koja se naziva **matrica kamere(C)**, i mi ćemo je koristiti za kalibraciju kamere.
Međutim, prave kamere ne koriste sitne rupice(**pinhole**), već koriste sočiva za fokusiranje na više svetlosnih zraka istovremeno, što im omogućava da brzo formiraju slike. Ali, sočiva takođe mogu uvesti distorziju.

# Vrste distorzija

**Radialna distorzija**: Radialna distorzija je najčešći tip koji utiče na slike, u kojem su fotografije snimljene pravilnim linijama kamerom izgledale blago zakrivljene ili savijene.
<br>
**Tangencijalna distorzija**: Tangencijalno izobličenje nastaje uglavnom zato što sočiva nisu paralelno poravnana sa ravninom slike, zbog čega se slika malo produžuje ili naginje, zbog čega se predmeti prikazuju dalje ili čak bliže nego što zapravo jesu.
<br>
<img src="assets/types_dist.png" width="800px" alt="Radially Distorted by a camera">
<p style="text-align: center;">Radialna distorzija</p>

Dakle, da bi se smanjila distorzija, koriste se **Koeficijenti distorzije**, čije vrednosti odražavaju količinu radijalne i tangencijalne distorzije na slici i **matrica kamere** koja sadrži unutrašnje parametre koji su specifični za kameru. Sadrži informacije poput fokusne dužine $(f_x, f_y)$ i optičke centre $(c_x, c_y)$.

<p style="text-align: center;"> $D = [k_1 k_2 p_1 p_2 k_3]$ </p>
<p style="text-align: center;"> $D = [k_1 k_2 p_1 p_2 k_3 k_4 s_{x1} s_{y1}]$ </p>
<p style="text-align: center;"> $$C =\begin{pmatrix}
f_x&0&c_x\\
0&f_y&c_y\\
0&0&1
\end{pmatrix},$$
 </p>

U ovom projektu se radilo sa već datim koeficijentima distorzije i matrice kamere, pa nema potrebe da se oni  nalaze. Distorziju je na prvi pogled teško uočiti kao što se može videti sa primera


<img src="assets/pic_1dis.JPG" width="800px">
<p style="text-align: center;">sa distorzijom</p>
<br>
<img src="assets/pic_1und.png" width="800px">
<p style="text-align: center;">bez distorzije</p>
<br>
Ali direktnim stavljanjem slike na sliku uočava se razlika:
<br>
<img src="assets/difference.gif" width="800px">


Dataset(slike) nad kojima je radjenp popravljanje distorzije je sa https://www.eth3d.net/, uz slike se i dobijaju informacije o distorziji i matrici kamere. Takodje date su formule tranformacija koje se vrše nad slikama:
    
1) Ako je z-koordinata tačke manja ili jednaka nuli, tačka se neposmatra.

2) Projekcija tačke na ravan virtuelne slike tako što se njene koordinate x i y podele sa z koordinatom, što rezultira nedistortovanim vrednostima x i y $(x_d, y_d)$ <br>
<p style="text-align: center;">
   $ x_d = X/W \\
    y_d = Y/W$
</p>
<br>
3) Primenjujemo model jednakih udaljenih radijalnih distorzija da bi dobili
<p style="text-align: center;">
   $ r = \sqrt{x_d^2 + y_d^2}\\
    \theta = tan^{-1}(r)
   $
    $$\begin{pmatrix}
            u_d\\
            v_d
        \end{pmatrix} =  {\theta \over r} \begin{pmatrix}
                                                x_d\\
                                                y_d
                                          \end{pmatrix} $$
</p>
<br>

4) Primenite dalju distorziju da bi dobili distorzovane normalizovane koordinate slike $(u_n, v_n)$
<p style="text-align: center;">
   $ t_r = 1 + k_1 \theta^2 + k_2 \theta^4 + k_3 \theta^6 + k_4 \theta^8\\
    u_n = u_d t_r + 2 p_1 u_d v_d + p_2(\theta^2 + 2 u_d^2) + s_{x1} \theta^2\\
    v_n = v_d t_r + 2 p_2 u_d v_d + p_1(\theta^2 + 2 v_d^2) + s_{y1} \theta^2\\
   $
</p>
<br>
5) Kovertovati u koordiante piksela $(u,v)$
<p style="text-align: center;">
   $
    u = f_x u_n + c_x\\
    v = f_y v_n + c_y\\
   $
</p>
<br>


# Implementacija

In [18]:
from matplotlib import pyplot as plt
import numpy as np

import os
import sys
import collections
import struct

import cv2

from skimage.transform import FundamentalMatrixTransform

### Pomoćna skripta za učitavanje podataka o kameri i slika

In [19]:
"""
Preuzeto sa https://www.eth3d.net/documentation
Pomocna skripta za ucitavanje podataka o kameri i slika
"""
CameraModel = collections.namedtuple(
    "CameraModel", ["model_id", "model_name", "num_params"])
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])

CAMERA_MODELS = {
    CameraModel(model_id=0, model_name="SIMPLE_PINHOLE", num_params=3),
    CameraModel(model_id=1, model_name="PINHOLE", num_params=4),
    CameraModel(model_id=2, model_name="SIMPLE_RADIAL", num_params=4),
    CameraModel(model_id=3, model_name="RADIAL", num_params=5),
    CameraModel(model_id=4, model_name="OPENCV", num_params=8),
    CameraModel(model_id=5, model_name="OPENCV_FISHEYE", num_params=8),
    CameraModel(model_id=6, model_name="FULL_OPENCV", num_params=12),
    CameraModel(model_id=7, model_name="FOV", num_params=5),
    CameraModel(model_id=8, model_name="SIMPLE_RADIAL_FISHEYE", num_params=4),
    CameraModel(model_id=9, model_name="RADIAL_FISHEYE", num_params=5),
    CameraModel(model_id=10, model_name="THIN_PRISM_FISHEYE", num_params=12)
}
CAMERA_MODEL_IDS = dict([(camera_model.model_id, camera_model) for camera_model in CAMERA_MODELS])

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

def read_cameras_text(path):
    cameras = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                camera_id = int(elems[0])
                model = elems[1]
                width = int(elems[2])
                height = int(elems[3])
                params = np.array(tuple(map(float, elems[4:])))
                cameras[camera_id] = Camera(id=camera_id, model=model,
                                            width=width, height=height,
                                            params=params)
    return cameras

def read_images_text(path):
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images

def read_points3D_text(path):
    points3D = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                point3D_id = int(elems[0])
                xyz = np.array(tuple(map(float, elems[1:4])))
                rgb = np.array(tuple(map(int, elems[4:7])))
                error = float(elems[7])
                image_ids = np.array(tuple(map(int, elems[8::2])))
                point2D_idxs = np.array(tuple(map(int, elems[9::2])))
                points3D[point3D_id] = Point3D(id=point3D_id, xyz=xyz, rgb=rgb,
                                               error=error, image_ids=image_ids,
                                               point2D_idxs=point2D_idxs)
    return points3D

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])


def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec


def read_model(path):
    cameras = read_cameras_text(os.path.join(path, "cameras.txt"))
    images = read_images_text(os.path.join(path, "images.txt"))
    points3D = read_points3D_text(os.path.join(path, "points3D.txt"))
    return cameras, images, points3D

In [9]:
folder = 'multi_view_training_dslr_jpg/'
image_set = 'courtyard/'
info_folder = folder + image_set + 'dslr_calibration_jpg/'

### Učitavanje slika i podataka pomoću skripte

In [11]:
camera_info, images_info, points_info = read_model(info_folder)
# Loading images in a list
images = list(list())
for dirName, subdirList, fileList in os.walk(folder + image_set + 'images/dslr_images/'):
    for fname in fileList:
        images.append(['dslr_images/' + fname, cv2.imread(dirName + fname)])

In [ ]:
"""
"Camera", ["id", "model", "width", "height", "params"]
"Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"]
"Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"]
"""

# Vršenje undistorcije slika

In [16]:
number_of_pictures = 1
destinations = list()
for image in images:
    
    if number_of_pictures < 1:
        break
    number_of_pictures -= 1
    
    # Fiding data about picture
    for i in images_info:
        if images_info[i].name == image[0]:
            image_data = images_info[i]
            break

    # Finding camera for given picture stored in camera_data
    for i in camera_info:
        if camera_info[i].id == image_data.camera_id:
            camera_data = camera_info[i]

    # Loading camera params and distortion coeff
    fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, sx1, sy1 = camera_data.params    

    # Grouping camera and distortion coeff
    dist_coeff = np.array([k1,k2,p1,p2,k3, k4,  sx1, sy1 ])
    cam_matx = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])
#     Rot = np.array(image_data.qvec)
#     Tran = np.array(image_data.tvec)
    
    print("Current picture is \'%s\'" % image[0])
    
    img = image[1]
    h,  w = img.shape[:2]
    
    # Used for comparison from lib cv2
    #####################################################################################################
    # new_camera_mtx, roi=cv2.getOptimalNewCameraMatrix(cam_matx,dist_coeff,(w,h),1,(w,h))
    # mapx, mapy = cv2.initUndistortRectifyMap(cam_matx, dist_coeff2, None, None, (w,h), cv2.CV_32FC1)
    # dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
    # cv2.imwrite('calibresultc_v2.png',dst)
    #####################################################################################################
    
    
    # Caluclating maps of X and Y pixels
    print("Caluclating maps....")
    myMpx, myMpy = initUndistortRectifyMap_fisheye(cam_matx, dist_coeff, (w,h))
    
    # Remaping new image with given maps of pixels
    print("Remaping image....")
    dst = cv2.remap(img, myMpx, myMpy, cv2.INTER_LINEAR)
    destinations.append(dst)
    
    print("Saving picture und_%s"%(image[0]))
    print(cv2.imwrite('undistort_resault_' + str(number_of_pictures) + '.png',dst))
    
    

Current picture is 'dslr_images/DSC_0311.JPG'
Caluclating maps....
Remaping image....
Saving picture und_dslr_images/DSC_0311.JPG
True


**cv2.remap**
Funkcija izvodi različite geometrijske transformacije 2D slika. Ona ne menja sadržaj slike, već deformiše rešetku piksela i mapiraju ovu deformisanu mrežu na odredišnu sliku. Zapravo, da bi se izbeglo uzorkovanje artefakata(previše izlaznih piksela zavisi od jednog ulaznog piksela.), mapiranje se vrši obrnutim redosledom, od odredišta do izvora. To jest, za svaki piksel (x, y) odredišne slike, funkcia izračunava koordinate odgovarajućeg piksela „donora“ u izvornoj slici i kopiraju vrednost piksela:
<br>
<p style="text-align: center;"> $dst(x,y) = src(f_x(x,y), f_y(x,y))$ </p>
<br>
U slučaju kada je odredjeno preslikavanje unapred $\left<g_x, g_y\right>: \texttt{src} \rightarrow \texttt{dst}$, funkcije OpenCV prvo izračunaju odgovarajuće obrnuto mapiranje $\left<f_x, f_y\right>: \texttt{dst} \rightarrow \texttt{src}$, a zatim iskoristite gornju formulu

In [13]:
def initUndistortRectifyMap_pinhole(camMat, dCoeff, size):
    """
    return map_x, map_y
    """
    
    w, h = size
    
    # making new camera matrix, but alpha caluclations are missing
    newCMat = camMat
    newCMat[0][2] = (w-1)*0.5
    newCMat[1][2] = (h-1)*0.5
    
    # u0 = cx , v0  = cy
    fx = camMat[0][0]
    fy = camMat[1][1]
    
    cx = camMat[0][2]
    cy = camMat[1][2]
    
    
    
    k1, k2, p1, p2, k3 = dCoeff
    k4 = k5 = k6 = 0
        
    mapx = np.ones((w,h))
    mapy = np.ones((w,h))
    
    mapx = mapx.astype('float32')
    mapy = mapy.astype('float32')
    

    for i in range(0,w):
        u = i
        x = (u - newCMat[0][2]) / newCMat[0][0]
        x2 = x*x
        for j in range(0,h):
            v = j

            y = (v - newCMat[1][2]) / newCMat[1][1]

            #Rot matrix currently identity
#             [x, y, w] = np.dot(np.linalg.inv([[1,0,0],[0,1,0],[0,0,1]]),np.transpose([x,y,1]))

            y2 = y*y;
            r2 = x2 + y2
            _2xy = 2*x*y;

            kr = (1 + ((k3*r2 + k2)*r2 + k1)*r2)/(1 + ((k6*r2 + k5)*r2 + k4)*r2);
            
            u = fx*(x*kr + p1*_2xy + p2*(r2 + 2*x2)) + cx;
            v = fy*(y*kr + p1*(r2 + 2*y2) + p2*_2xy) + cy;

            mapx[i][j] = u
            mapy[i][j] = v

    return np.transpose(mapx), np.transpose(mapy)


In [14]:

def initUndistortRectifyMap_fisheye(camMat, dCoeff, size):
    """
    return map_x, map_y
    """
    
    w, h = size
    
    # making new camera matrix, but alpha caluclations are missing
    newCMat = camMat
    newCMat[0][2] = (w-1)*0.5
    newCMat[1][2] = (h-1)*0.5
    

    fx = camMat[0][0]
    fy = camMat[1][1]
    
    cx = camMat[0][2]
    cy = camMat[1][2]
    
    k1, k2, p1, p2, k3, k4, sx1, sx2 = dCoeff
        
    mapx = np.ones((w,h)).astype('float32')
    mapy = np.ones((w,h)).astype('float32')
    
#     mapx = mapx.astype('float32')
#     mapy = mapy.astype('float32')
    

    for i in range(0,w):
        u = i
        x = (u - newCMat[0][2]) / newCMat[0][0]
        x2 = x*x
        for j in range(0,h):
            v = j

            y = (v - newCMat[1][2]) / newCMat[1][1]

            #Rot matrix currently identity
#           [x, y, w] = np.dot(np.linalg.inv([[1,0,0],[0,1,0],[0,0,1]]),np.transpose([x,y,1]))

            y2 = y*y;
            r2 = x2 + y2
            
            tao = np.arctan(np.sqrt(r2))
            tao2 = tao**2
            
            ud = tao/np.sqrt(r2) * x
            vd = tao/np.sqrt(r2) * y

#             tr = 1 + k1*tao**2 + k2*tao**4 + k3*tao**6 + k4 * tao**8
            tr = 1 + tao2*(k1 + tao2*(k2 + tao2*(k3 + tao2*k4)))

    
            u = fx*(ud*tr + 2*p1*ud*vd+p2*(tao**2+2*ud**2)+sx1*tao**2) + cx;
            v = fy*(vd*tr + 2*p2*ud*vd+p1*(tao**2+2*vd**2)+sy1*tao**2) + cy;

            mapx[i][j] = u
            mapy[i][j] = v

    return np.transpose(mapx), np.transpose(mapy)

Nakon izvšavanja koda dobijaju se neke od sledećih slika:

<img src="assets/undistort_resault_1.png" width="800px">
<img src="assets/distort_resault_1.JPG" width="800px">


<img src="assets/undistort_resault_2.png" width="800px">
<img src="assets/distort_resault_2.JPG" width="800px">
